In [1]:
import os
import numpy as np
import pandas as pd
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import train_test_split
from surprise import KNNBasic
from surprise import accuracy
import random
import pickle

#Para garantizar reproducibilidad en resultados
seed = 10
random.seed(seed)
np.random.seed(seed)

In [2]:
if not (os.path.exists('./Data/preprocessed_user_item_rating.csv')):
  raise ValueError('El archivo preprocessed_user_item_rating.csv no fue encontrado en el path')
else:
  print("El archivo ha sido cargado")

El archivo ha sido cargado


In [3]:
ratings=pd.read_csv('./Data/preprocessed_user_item_rating.csv', sep = ',', header=0, names = [ 'userid', 'artist-name', 'rating' ] )
ratings = ratings.loc[:,['userid', 'artist-name','rating']]
ratings

,userid,artist-name,rating
0,user_001000,Wilco,5.0
1,user_001000,Radiohead,4.9
2,user_001000,Animal Collective,4.7
3,user_001000,Girl Talk,4.6
4,user_001000,Aesop Rock,4.2
...,...,...,...
896880,user_000001,Jamie Lidell,0.2
896881,user_000001,Nick Holmes,0.2
896882,user_000001,Nuyorican Soul,0.2
896883,user_000001,The Birthday,0.2


# Creacion sistema de recomendacion

In [4]:
reader = Reader( rating_scale = ( 1, 10 ) )
#Se crea el dataset a partir del dataframe
surprise_dataset = Dataset.load_from_df( ratings[ [ 'userid', 'artist-name', 'rating' ] ], reader )

In [5]:
trainset, testset=  train_test_split(surprise_dataset, test_size=.2)

### Modelo basado en distancias coseno

In [6]:
sim_options = {'name': 'cosine',
               'user_based': False  # calcule similitud item-item
               }
algo = KNNBasic(k=30, min_k=5, sim_options=sim_options)

In [7]:
predictions = algo.fit(trainset).test(testset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [8]:
pickle.dump( predictions, open( "./Data/predictions_ii_cosine.p", "wb" ) )

### Modelo basado en índice de Jaccard

### Modelo basado en correlación de Pearson

In [9]:
sim_options = {'name': 'pearson_baseline',
               'user_based': False,
               'shrinkage': 0  # no shrinkage
               }
algo = KNNBasic(sim_options=sim_options)

In [10]:
predictions = algo.fit(trainset).test(testset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [11]:
pickle.dump( predictions, open( "./Data/predictions_ii_pearson.p", "wb" ) )

## Recomendaciones

In [26]:
pickle_predictions = pickle.load( open( "./Data/predictions_ii_cosine.p", "rb" ) )

In [27]:
#Predicciones para usuario user_001000
user_predictions=list(filter(lambda x: x[0]=='user_001000',pickle_predictions))

In [28]:
#Ordenamos de mayor a menor estimación de relevancia
user_predictions.sort(key=lambda x : x.est, reverse=True)

In [29]:
#tomamos las 10 primeras predicciones
user_predictions=user_predictions[0:10]

In [30]:
#Se convierte a dataframe
labels = ['artist', 'estimation']
df_predictions = pd.DataFrame.from_records(list(map(lambda x: (x.iid, x.est) , user_predictions)), columns=labels)

### Recomendaciones usuario 001000

In [31]:
df_predictions.loc[:,['artist']]

,artist
0,Murder By Death
1,Thunderbirds Are Now!
2,Groove Collective
3,Rilo Kiley
4,Ludachrist
5,Ben Folds
6,Yeasayer
7,Santogold
8,Canada
9,El Ten Eleven


In [32]:
accuracy.rmse( user_predictions, verbose = True )

RMSE: 0.5108


0.5107565617045281